In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from encode_data import *
from midi_data import *

In [3]:
from tqdm import tqdm

## Encode music21 stream 
This notebook uses a simple format 
- no measure separators, instruments, or separated octaves
- Format: noteWithOctave, action type
- note repr: nG#4 t1/t2

### Load midi data

In [4]:
path = Path('data/midi')
csv_path = path/'metadata'

In [5]:
source_dir = 'midi_transform_v1'
out_dir = 'midi_transcribe_v1_simple'
source_csv = csv_path/f'{source_dir}.csv'
out_csv = csv_path/f'{out_dir}.csv'

In [6]:
def simple_fmt(note):
    nname = NPRE + note.pitch.nameWithOctave
    tname = TSTART if note.tie == VALTSTART else TCONT # ts=note start, tc=note continue
    return [nname,tname]

separate_measures = False

In [7]:
import pandas as pd

In [8]:
df = pd.read_csv(source_csv, index_col=0); df.head()

/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (6,11,13,16,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,midi,bpm,source,instruments,artist,song_url,title,inferred_offset,ht_offset,genres,...,seconds,ht_key,time_signature,ht_bpm,parts,inferred_key,ht_time_signature,ht_mode,midi_title,midi_transform_v1
0,data/midi/midi_sources/hooktheory/pianoroll/w/...,128.0,hooktheory,"Piano,Piano",wayne-sharpe,https://www.hooktheory.com/theorytab/view/wayn...,yu-gi-oh-theme-song,0.0,0.0,NaN,...,15.000000,C,4/4,128.0,"intro,chorus",C major,4.0,1.0,yu-gi-oh3,data/midi/midi_transform_v1/hooktheory/pianoro...
1,data/midi/midi_sources/hooktheory/pianoroll/w/...,NaN,hooktheory,NaN,weezer,https://www.hooktheory.com/theorytab/view/weez...,beverly-hills,NaN,0.0,NaN,...,NaN,C,NaN,128.0,intro-and-verse,NaN,4.0,NaN,My New Song,NaN
2,data/midi/midi_sources/hooktheory/pianoroll/w/...,108.0,hooktheory,"Piano,Piano",weezer,https://www.hooktheory.com/theorytab/view/weez...,falling-for-you,-3.0,-3.0,NaN,...,11.111111,Eb,4/4,108.0,"intro,verse,chorus,solo",E- major,4.0,1.0,falling for you intro,data/midi/midi_transform_v1/hooktheory/pianoro...
3,data/midi/midi_sources/hooktheory/pianoroll/w/...,121.0,hooktheory,"Piano,Piano",weezer,https://www.hooktheory.com/theorytab/view/weez...,buddy-holly,4.0,4.0,"Pop,Rock",...,43.388430,Ab,4/4,121.0,"verse,pre-chorus,chorus,bridge,solo",A- major,4.0,1.0,Buddy Holly,data/midi/midi_transform_v1/hooktheory/pianoro...
4,data/midi/midi_sources/hooktheory/pianoroll/w/...,180.0,hooktheory,"Piano,Piano",wavves,https://www.hooktheory.com/theorytab/view/wavv...,dreams-of-grandeur,-2.0,3.0,NaN,...,21.333333,A,4/4,180.0,verse,B minor,4.0,1.0,dreams of grandeur,data/midi/midi_transform_v1/hooktheory/pianoro...


In [9]:
df_filtered = df.loc[df['time_signature'] == '4/4']; df_filtered.shape

(30201, 21)

In [10]:
from data_sources import process_parallel

In [11]:
def transcribe_file(idxrow):
    idx,row = idxrow
    midi_file = row[source_dir]
    if not isinstance(midi_file, str) or not Path(midi_file).exists(): return idx,None
    out_file = Path(midi_file.replace(f'/{source_dir}/', f'/{out_dir}/')).with_suffix('.txt')
    out_file.parent.mkdir(parents=True, exist_ok=True)
    if out_file.exists(): return idx,str(out_file)
    try:
        seq = midi2seq(midi_file)
        seq_comp = remove_seq_rests(trim_seq_rests(seq))
        delta_trim = len(seq) - len(seq_comp)
        if delta_trim > 100: print(f'Removed {delta_trim} rests from {midi_file}')
        string_repr = seq2str(seq_comp, note_func=simple_fmt, 
                              separate_measures=separate_measures,
                             )
        with open(out_file, 'w') as tf:
            tf.write(string_repr)
    except Exception as e:
        print('Error converting midi to sequence:', e)
        return idx,None
    return idx,str(out_file)

In [12]:
process_parallel(transcribe_file, df_filtered.iterrows(), total=df_filtered.shape[0])

Compressing rests: 24 -> 16
Compressing rests: 24 -> 16
Compressing rests: 24 -> 16
Compressing rests: 24 -> 16
Compressing rests: 24 -> 16
Removed 134 rests from data/midi/midi_transform_v1/hooktheory/pianoroll/j/jacob-mann/kogi/intro_key.mid
Removed 101 rests from data/midi/midi_transform_v1/hooktheory/pianoroll/j/john-williams/the-raiders-march---indiana-jones-theme/intro-and-verse_key.mid
Compressing rests: 32 -> 16
Compressing rests: 28 -> 16
Compressing rests: 24 -> 16
Compressing rests: 52 -> 16
Compressing rests: 24 -> 16
Removed 129 rests from data/midi/midi_transform_v1/hooktheory/pianoroll/b/boyinaband/lets-summon-the-devil/instrumental_key.mid
Removed 136 rests from data/midi/midi_transform_v1/hooktheory/pianoroll/b/bruno-mars/uptown-funk/intro-and-verse_key.mid
Compressing rests: 24 -> 16
Compressing rests: 24 -> 16
Removed 153 rests from data/midi/midi_transform_v1/hooktheory/pianoroll/d/dawid-podsiadlo/trojkaty-i-kwadraty/verse_key.mid
Removed 129 rests from data/midi/mi

Compressing rests: 22 -> 18
Compressing rests: 22 -> 18
Compressing rests: 22 -> 18
Compressing rests: 22 -> 18
Compressing rests: 22 -> 18
Removed 322 rests from data/midi/midi_transform_v1/freemidi/genre-dance-eletric/Depeche Mode - Edge To Life.mid
Compressing rests: 64 -> 16
Compressing rests: 128 -> 16
Removed 171 rests from data/midi/midi_transform_v1/freemidi/genre-dance-eletric/Depeche Mode - Control Freak.mid
Compressing rests: 134 -> 18
Compressing rests: 60 -> 16
Removed 410 rests from data/midi/midi_transform_v1/freemidi/genre-dance-eletric/Madonna - American Pie.mid
Removed 1133 rests from data/midi/midi_transform_v1/freemidi/genre-dance-eletric/Moby - The Blue Terrors Of Lawns.mid
Compressing rests: 128 -> 16
Removed 113 rests from data/midi/midi_transform_v1/freemidi/genre-dance-eletric/New Order - Subculture.mid
Removed 124 rests from data/midi/midi_transform_v1/freemidi/genre-dance-eletric/Tori Amos - Cloud On My Tongue.mid
Compressing rests: 47 -> 19
Compressing rests

Compressing rests: 286 -> 18
Compressing rests: 367 -> 19
Compressing rests: 335 -> 19
Compressing rests: 334 -> 18
Compressing rests: 30 -> 18
Compressing rests: 368 -> 16
Compressing rests: 335 -> 19
Compressing rests: 30 -> 18
Compressing rests: 335 -> 19
Compressing rests: 30 -> 18
Compressing rests: 30 -> 18
Compressing rests: 30 -> 18
Compressing rests: 351 -> 19
Compressing rests: 30 -> 18
Compressing rests: 30 -> 18
Compressing rests: 335 -> 19
Compressing rests: 30 -> 18
Compressing rests: 335 -> 19
Compressing rests: 30 -> 18
Compressing rests: 30 -> 18
Compressing rests: 30 -> 18
Compressing rests: 351 -> 19
Compressing rests: 30 -> 18
Compressing rests: 335 -> 19
Compressing rests: 335 -> 19
Compressing rests: 367 -> 19
Compressing rests: 335 -> 19
Compressing rests: 335 -> 19
Compressing rests: 367 -> 19
Compressing rests: 335 -> 19
Compressing rests: 335 -> 19
Compressing rests: 367 -> 19
Compressing rests: 335 -> 19
Compressing rests: 30 -> 18
Compressing rests: 30 -> 18

Removed 1937 rests from data/midi/midi_transform_v1/freemidi/genre-dance-eletric/Kylie Minogue - Can't Get You out of My Head (3).mid
Removed 128 rests from data/midi/midi_transform_v1/freemidi/genre-dance-eletric/Craig David - 7 Days.mid
Compressing rests: 124 -> 16
Removed 601 rests from data/midi/midi_transform_v1/freemidi/genre-dance-eletric/Pitbull - Give Me Everything.mid
Removed 329 rests from data/midi/midi_transform_v1/freemidi/genre-dance-eletric/Toni Basil - Mickey.mid
Compressing rests: 39 -> 19
Compressing rests: 39 -> 19
Removed 146 rests from data/midi/midi_transform_v1/freemidi/genre-dance-eletric/DJ Bobo - Let The Dream Come True.mid
Compressing rests: 64 -> 16
Removed 453 rests from data/midi/midi_transform_v1/freemidi/genre-dance-eletric/Tatu - How Soon Is Now.mid
Compressing rests: 64 -> 16
Removed 108 rests from data/midi/midi_transform_v1/freemidi/genre-dance-eletric/Depeche Mode - A Question Of Lust.mid
Compressing rests: 48 -> 16
Compressing rests: 124 -> 16
Rem

KeyboardInterrupt: 

Removed 867 rests from data/midi/midi_transform_v1/freemidi/genre-dance-eletric/Depeche Mode - Any Second Now.mid
Removed 148 rests from data/midi/midi_transform_v1/freemidi/genre-dance-eletric/Alex Party - Don't Give Me Your Life.mid
Removed 174 rests from data/midi/midi_transform_v1/freemidi/genre-dance-eletric/Cher - Half Breed.mid
Removed 113 rests from data/midi/midi_transform_v1/freemidi/genre-dance-eletric/Alanis Morissette - You Oughta Know.mid
Removed 245 rests from data/midi/midi_transform_v1/freemidi/genre-dance-eletric/2 Unlimited - No Limits.mid
Compressing rests: 58 -> 18
Removed 105 rests from data/midi/midi_transform_v1/freemidi/genre-dance-eletric/Gloria Estefan - 1-2-3.mid
Compressing rests: 32 -> 16
Removed 319 rests from data/midi/midi_transform_v1/freemidi/genre-dance-eletric/New Order - Love Vigilantes.mid
Removed 118 rests from data/midi/midi_transform_v1/freemidi/genre-dance-eletric/Bjork - Come To Me Remix.mid
Compressing rests: 280 -> 16
Compressing rests: 154

In [ ]:
tdf = pd.DataFrame(data={out_dir: list(transcribed_files.values())}, index=list(transcribed_files.keys()))

In [ ]:
merged_df = df.join(tdf, how='outer'); tdf.shape, df.shape, merged_df.shape

In [ ]:
merged_df.to_csv(out_csv); merged_df.head()